In [1]:
from collections import namedtuple
# other packages
import pandas as pd
from allennlp.predictors import SimpleSeq2SeqPredictor, Seq2SeqPredictor
import tqdm
# pytorch
import torch
import torch.optim as optim
# preprocessing
from imblearn.under_sampling import RandomUnderSampler
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.vocabulary import Vocabulary
# model
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.seq2seq_encoders import StackedSelfAttentionEncoder
# attention
from allennlp.modules.attention import AdditiveAttention
# encoder
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.training.trainer import Trainer
from sklearn.model_selection import train_test_split

In [2]:
torch.cuda.is_available()

True

In [3]:
Config = namedtuple('Config', [
                    'lazy',
                    'max_vocab_size', 
                    'batch_size', 
                    'epochs', 
                    'max_seq_len', 
                    'IN_EMBEDDING_DIM', 
                    'HIDDEN_DIM', 
                    'OUT_EMBEDDING_DIM', 
                    'CUDA_DEVICE',
                    'n_samples'
])

# create config file

In [7]:
config = Config(False,
               20000,
               16,
               10,
               10,
               64,
               32,
               32,
               0 if torch.cuda.is_available() else -1, 
               60000)

# prepare data

In [5]:
raw_dataset = pd.read_csv('data/ru_train.csv')

raw_dataset['before'] = raw_dataset['before'].astype(str)
raw_dataset['after'] = raw_dataset['after'].astype(str)

d = raw_dataset['class'].value_counts().to_dict()

for i in d:
    if d[i] > config.n_samples:
        d[i] = config.n_samples
        


rus = RandomUnderSampler(sampling_strategy=d, random_state=0)

raw_dataset_resampled, _ = rus.fit_resample(raw_dataset, raw_dataset['class'])

In [6]:
df = raw_dataset_resampled[['before', 'after']]

df_train, df_test = train_test_split(df, test_size=0.1)

df_train.to_csv('train_dataset.tsv', index=False, header=False, sep='\t')
df_test.to_csv('test_dataset.tsv', index=False, header=False, sep='\t')

# create reader

In [8]:
reader = Seq2SeqDatasetReader(
    source_tokenizer = WordTokenizer(),
    target_tokenizer = WordTokenizer(),
    source_token_indexers={'tokens': SingleIdTokenIndexer()},
    target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')},
    lazy=config.lazy
)

train_dataset = reader.read('train_dataset.tsv')
validation_dataset = reader.read('test_dataset.tsv')

431636it [01:43, 4171.00it/s]
47960it [00:10, 4528.49it/s]


# prepare vocabulary

In [9]:
vocab = Vocabulary.from_instances(train_dataset,
                                  min_count={'tokens': 3, 'target_tokens': 3},
                                  max_vocab_size=config.max_vocab_size
                                 )

in_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=config.IN_EMBEDDING_DIM
                        )

source_embedder = BasicTextFieldEmbedder({"tokens": in_embedding})

iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("source_tokens", "num_tokens")],
                         )

iterator.index_with(vocab)

100%|██████████| 431636/431636 [00:02<00:00, 175319.02it/s]


# prepare model

In [10]:
## get simple encoder
encoder = StackedSelfAttentionEncoder(input_dim=config.IN_EMBEDDING_DIM, 
                                      hidden_dim=config.HIDDEN_DIM, 
                                      projection_dim=32, 
                                      feedforward_hidden_dim=32, 
                                      num_layers=1, 
                                      num_attention_heads=8)

In [11]:
max_decoding_steps = 20

model = SimpleSeq2Seq(vocab, 
                      source_embedder, 
                      encoder, 
                      max_decoding_steps,
                      target_embedding_dim=config.OUT_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      beam_size=8,
                      use_bleu=False,
                      scheduled_sampling_ratio = 0.15)

In [12]:
if torch.cuda.is_available():
    model.cuda(config.CUDA_DEVICE)

In [13]:
optimizer = optim.Adam(model.parameters())

# training

In [14]:
torch.cuda.is_available()

True

In [15]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  num_epochs=config.epochs,
                  cuda_device=config.CUDA_DEVICE,
                  patience=2)

In [ ]:
print(f'Will train for {config.epochs} epochs')
trainer.train()

  0%|          | 0/26978 [00:00<?, ?it/s]

Will train for 10 epochs


loss: 2.0695 ||: 100%|██████████| 26978/26978 [04:43<00:00, 95.01it/s] 
  0%|          | 0/2998 [00:00<?, ?it/s]/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
loss: 1.4605 ||: 100%|██████████| 2998/2998 [00:28<00:00, 106.62it/s]
loss: 1.5699 ||: 100%|██████████| 26978/26978 [04:41<00:00, 95.89it/s] 
loss: 1.2985 ||: 100%|██████████| 2998/2998 [00:26<00:00, 113.05it/s]
loss: 1.4626 ||: 100%|██████████| 26978/26978 [04:34<00:00, 98.33it/s] 
loss: 1.2217 ||: 100%|██████████| 2998/2998 [00:28<00:00, 106.51it/s]
loss: 1.4016 ||: 100%|██████████| 26978/26978 [04:40<00:00, 96.11it/s] 
loss: 1.1591 ||: 100%|██████████| 2998/2998 [00:27<00:00, 108.14it/s]
loss: 1.3532 ||: 100%|██████████| 26978/26978 [04:35<00:00, 97.81it/s] 
loss: 1.1249 ||: 100%|██████████| 2

In [ ]:
with open("word_emb.th", 'wb') as f:
    torch.save(model.state_dict(), f)
    
tqdm.tqdm.pandas()

predictor = SimpleSeq2SeqPredictor(model, reader)

In [ ]:


kaggle_test = pd.read_csv('data/ru_test_2.csv')
kaggle_test['before'] = kaggle_test['before'].astype(str)

small_kaggle = pd.DataFrame(kaggle_test['before'].unique(), columns=['before'])


In [ ]:
small_kaggle['after'] = small_kaggle['before'].progress_apply(lambda x: ''.join(predictor.predict(x)['predicted_tokens']))

In [ ]:
small_kaggle = small_kaggle.set_index('before')

In [ ]:
kaggle_test = kaggle_test.join(small_kaggle, on='before')
kaggle_test['id'] = kaggle_test['sentence_id'].astype(str)+'_'+kaggle_test['token_id'].astype(str)

In [ ]:
kaggle_test[['id', 'after']].to_csv('sub.csv', index=False)